# Notebook 13: Hubs and Authorities
***

In this notebook we'll see the HITS algorithm in action, and examine a crude approach to generating graphs stochastically. We will also confirm what we saw in the lecture, which is that the hub and the authority scores are the principle eigenvalues of some forms of the link matrix describing the graph.

We'll need numpy for this notebook, so let's load it.

In [1]:
import numpy as np

<br>

### Exercise 0: Following along with lecture is cool!

From the lecture example, here is out graph:

<img width=200px src="http://www.cs.colorado.edu/~anwo7157/home/resources/hubsauthorities1.png">

And the associated link matrix:

In [2]:
L = np.array([[0,1,1,1,0],
              [1,0,0,1,0],
              [0,0,0,0,1],
              [0,1,1,0,0],
              [0,0,0,0,0]])

Since we are using max-component = 1 normalization, we can initialize our hub and authority scores as all 1s:

In [3]:
n = L.shape[0]
hub = np.ones(n)
aut = np.ones(n)

In the first step in the first iteration of the HITS algorithm, we update the authority scores as $a^{(1)} = L^T h^{(0)}$, then normalize $a^{(1)}$ so its largest component is equal to 1:

In [4]:
# step forward:
aut = np.matmul(np.transpose(L), hub)
# normalize:
aut = aut/np.max(aut)
print(aut)

[0.5 1.  1.  1.  0.5]


But hold on a second! Do you really want to keep transposing the $L$ matrix each iteration? That's for *suckers*. So let's save that in memory and never transpose it again.

In [5]:
LT = np.transpose(L)

Phew. We can now finish off the rest of the first iteration by updating and normalizing the hub scores as $h^{(1)} = L a^{(1)}$.

In [6]:
# step forward:
hub = np.matmul(L, aut)
hub = hub/np.max(hub)
print(hub)

[1.         0.5        0.16666667 0.66666667 0.        ]


Check that what we computed matches the by-hand calculation from lecture:

In [7]:
print("Authority scores: ", np.round(aut,4))
print("Hub scores: ", np.round(hub,4))

Authority scores:  [0.5 1.  1.  1.  0.5]
Hub scores:  [1.     0.5    0.1667 0.6667 0.    ]


Your mission, should you choose to accept it (spoiler alert: you should), is to iterate until the $L_2$ norm of both $h_{new} - h_{old}$ and $a_{new} - a_{old}$ are less than 0.001. (This means you will need to keep track of the old hub and authority scores within the iteration, for comparison against the new ones.)

For a sanity check, continue onward to Exercise 1!

In [8]:
def dist_L2(x,y):
    return np.sqrt(np.sum((x-y)**2))

n = L.shape[0]
hub = np.ones(n)
aut = np.ones(n)
tol = 0.001

hub_old = np.zeros(n) # initialize to track change in h and a
aut_old = np.zeros(n)
max_change = np.max([dist_L2(hub,hub_old), dist_L2(aut,aut_old)])
while max_change > tol:
    aut = np.matmul(np.transpose(L), hub)
    aut = aut/np.max(aut)
    hub_old = hub.copy()
    hub = np.matmul(L, aut)
    hub = hub/np.max(hub)
    max_change = np.max([dist_L2(hub,hub_old), dist_L2(aut,aut_old)])
    aut_old = aut.copy()
    print("Authority scores: ", np.round(aut,4))
    print("Hub scores: ", np.round(hub,4))
# TODO -- the rest of your HITS code (the main iteration)
#         goes down here!

Authority scores:  [0.5 1.  1.  1.  0.5]
Hub scores:  [1.     0.5    0.1667 0.6667 0.    ]
Authority scores:  [0.3 1.  1.  0.9 0.1]
Hub scores:  [1.     0.4138 0.0345 0.6897 0.    ]
Authority scores:  [0.2449 1.     1.     0.8367 0.0204]
Hub scores:  [1.     0.3813 0.0072 0.705  0.    ]
Authority scores:  [0.2236 1.     1.     0.8101 0.0042]
Hub scores:  [1.     0.3679 0.0015 0.7117 0.    ]
Authority scores:  [2.149e-01 1.000e+00 1.000e+00 7.991e-01 9.000e-04]
Hub scores:  [1.000e+00 3.623e-01 3.000e-04 7.145e-01 0.000e+00]
Authority scores:  [2.113e-01 1.000e+00 1.000e+00 7.946e-01 2.000e-04]
Hub scores:  [1.000e+00 3.599e-01 1.000e-04 7.157e-01 0.000e+00]
Authority scores:  [0.2098 1.     1.     0.7926 0.    ]
Hub scores:  [1.     0.359  0.     0.7162 0.    ]
Authority scores:  [0.2092 1.     1.     0.7919 0.    ]
Hub scores:  [1.     0.3585 0.     0.7164 0.    ]
Authority scores:  [0.2089 1.     1.     0.7915 0.    ]
Hub scores:  [1.     0.3584 0.     0.7165 0.    ]


<br>

### Exercise 1:  Hubbiness and Authorityness

In the lecture, we saw that the final converged values for the hub score ($h$) and the authority score ($a$) are the principle eigenvectors of the matrices $LL^T$ and $L^T L$, respectively. We can obtain the eigenvalues and eigenvectors of $LL^T$, for example, using the [numpy linalg.eig method](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html):

In [9]:
evals, evecs = np.linalg.eig(np.matmul(L,LT))

Just by having a look at the eigenvalues array, we notice that they are not necessarily sorted:

In [10]:
print(evals)

[4.79128785 0.20871215 2.         1.         0.        ]


The largest one is indeed first, so the principle eigenvector is the first column of `evecs`. But this may not be the case in general, so let's find the index of the maximal eigenvalue:

In [11]:
idx_princ = list(evals).index(np.max(evals))
print(idx_princ)

0


And use that to grab the principle eigenvector:

In [12]:
print(evecs[:,idx_princ])

[0.78045432 0.27960367 0.         0.55920734 0.        ]


*Hmm!* you say to yourself, *That doesn't match the hub score we found in Exercise 0. What have I done?!*

Don't even worry though! There is a good and simple explanation for it. Think about how we compute the hub score, and perhaps check the documentation for `linalg.eig` on how the eigenvector matrix is returned, and see if you can figure out the proper way to compare the two.

Then, compute the eigenvectors and eigenvalues associated with the $L^T L$ matrix and use them to check our work for the authority score. If you run into trouble, make sure that the first component of our eigenvectors is always nonnegative. If it's negative, you can flip the direction of a vector by multiplying by -1.

In [13]:
# SOLUTION:

# hub score:

# authority score:

In [14]:
# Crap! Something has gone terribly wrong. 
# It's because the first component is negative.
# So, we swap directions:


# TODO: fix that the first component is negative

<br>

### Exercise 2: Generating a random graph

The following code will generate a random graph on 6 nodes such that for each pair of nodes $(i,j)$, there is a connection from $i$ to $j$ with probability $p=0.33$.

In [15]:
np.random.seed(4022)
n = 6
p = 0.33
L1 = np.zeros((n,n))
for irow in range(n):
    for icol in range(n):
        L1[irow, icol] = np.random.choice([0,1], p=[1-p, p])
print(L1)

[[0. 1. 1. 0. 1. 1.]
 [1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1.]]


**[1]** On a piece of paper, draw this directed graph. Formulate a hypothesis about which node(s) will have the highest hub score, and which will have the highest authority score. What about the structure of the graph supports your hypotheses?

**Solution:**


**[2]** Suppose we want to construct a graph so it is relatively more likely that a particular node has a high hub score. How could we modify the above code, without hard-coding any specifc connections? Similarly, what if we wanted to make it more likely that a node has a higher authority score? Implement your modification so that Node 2 is more likely to be a hub, and Node 6 is more likely to be an authority. Save the resulting link matrix as `L2`.

In [16]:
# SOLUTION:

np.random.seed(4022)
n = 6
p = 0.33
L2 = np.zeros((n,n))
ihub = 1 # row of the hub
iaut = 5 # col of the authority

#TODO: fill the rest of the matrix where this hub/authority pair are more likely.

<br>

### Exercise 3: HITS-ing that graph

Verify your hypotheses from Exercise 2 on the given graph regarding the nodes that will be good hubs or authorities for your graphs. Use either the HITS algorithm (from Exercise 0) or the eigenvector approach (from Exercise 1). Did we succeed in our mission to make Node 2 a hub and Node 6 an authority?

In [17]:
# SOLUTION:

**Solution, Part 1:**

As we hypothesized, Node 1 has the highest hub score.

In [18]:
# SOLUTION:

**Solution, Part 2:**
